# Estimation of gradient index n2 from measured focal range of GRIN-rod F

In [ ]:
import numpy as np
import sympy as sym
import math
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.size': 16,
    'legend.frameon': True
})

## Direct calculation of F from known n2

### Full formula

The formula is initialy taken from the Yariv's 'Quantun electronics' p.113.

In [ ]:
n0 = 1
n2 = np.linspace(0.1, 100, 100)
fig, ax = plt.subplots(figsize=(10,6))
ax.set_xlabel('Gradient constant $n_2 \;\; (1/m^2)$')
ax.set_ylabel('Focus range, m')
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid(True, linestyle=':')
for L in [0.001, 0.01, 0.1]:
    F = 1/n0*np.sqrt(n0/n2)*np.cos(np.sqrt(n2/n0)*L)/np.sin(np.sqrt(n2/n0)*L)
    ax.plot(n2, F, label=f'L={L}')
ax.legend(title='Rod length, m')

### Precision of approximation

Approximation is done by means that `sin(x) = x` and `cos(x) = 1` when `x` is small.

In [ ]:
n2 = np.linspace(0.1, 100, 10)
fig, ax = plt.subplots(figsize=(10,6))
ax.set_xlabel('Gradient constant $n_2 \;\; (1/m^2)$')
ax.set_ylabel('Approximation error, %')
ax.grid(True, linestyle=':')
for L in [0.01, 0.05, 0.1]:
    F1 = 1/n0*np.sqrt(n0/n2)*np.cos(np.sqrt(n2/n0)*L)/np.sin(np.sqrt(n2/n0)*L)
    F2 = 1/L/n2
    ax.plot(n2, (F2-F1)/F1*100, label=f'L={L}')
ax.legend(title='Rod length, m')

## Inverse calculation of n2 from measured F 

In [ ]:
L = 0.1
F = 1.45

def equat_float(x):
    return x*F*n0*math.tan(x) - L

def equat_np(x):
    return x*F*n0*np.tan(x) - L
    
def calc_F(n2):
    return 1/n0*math.sqrt(n0/n2)/math.tan(math.sqrt(n2/n0)*L)

In [ ]:
# Calculation from reduced formula

x = sym.Symbol('x')
x1 = [sym.nsolve(x*F*n0*sym.sin(x) - L*sym.cos(x), x0) for x0 in [0, 0.1, 0.5, 1]]
print('Possible solutions for x:', x1)
x_root = np.min([x for x in x1 if x > 0])
n2_root = n0*(x_root/L)**2
F1 = calc_F(n2_root)
print('Solution: x_root =', x_root, 'n2 =', n2_root)
print(f'Back calculated F={F1}, error={(F1-F)/F*100}%')

In [ ]:
# Calculation from full formula

x = sym.Symbol('x')
x1 = [sym.nsolve(sym.sqrt(n0/x)/n0*sym.cos(sym.sqrt(x/n0)*L) - F*sym.sin(sym.sqrt(x/n0)*L), x0) for x0 in [0.1, 0.5, 1]]
print('Possible solutions for n2:', x1)
n2 = np.min([x for x in x1 if x > 0])
F1 = calc_F(n2)
print('Solution: n2 =', n2)
print(f'Diff with prev solution: diff_n2={abs(n2-n2_root)}')
print(f'Back calculated F={F1}, error={(F1-F)/F*100}%')

In [ ]:
# Calculate manually

x = np.linspace(0, 0.3, 100)
y = equat_np(x)
print('y_min =', np.min(y))
print('y @ x_root', equat_float(x_root))
fig, ax = plt.subplots(figsize=(10,6))
ax.grid(True, linestyle=':')
ax.plot(x, y)
ax.plot([np.min(x), np.max(x)], [0, 0])
ax.plot([x_root, x_root], [np.min(y), np.max(y)])

In [ ]:
x_next = 0
x_prev = 0
y_next = equat_float(x_next)
iters1 = 0
while y_next < 0:
    iters1 += 1
    x_prev = x_next
    x_next += 0.1
    y_next = equat_float(x_next)
#print(f'x_prev={x_prev}, x_next={x_next}, y_next={y_next}')
y_mid = y_next
iters2 = 0
while abs(y_mid) > 1e-8:
    iters2 += 1
    x_mid = (x_prev + x_next) / 2
    y_mid = equat_float(x_mid)
    #print(f'x_mid={x_mid}, y_mid={y_mid}, y_next={y_next}')
    if y_next > 0 and y_mid < 0:
        x_prev = x_mid
    else:
        x_next = x_mid
        y_next = y_mid
n2 = n0*(x_mid/L)**2
F1 = calc_F(n2)
print(f'Solution: x_mid={x_mid}, y_mid={y_mid}, n2={n2}, iters1={iters1}, iters2={iters2}, iters={iters1+iters2}')
print(f'Back calculated F={F1}, error={abs(F1-F)/F*100}%')
print(f'Diff with sym solution: diff_x={abs(x_mid-x_root)}, diff_n2={abs(n2-n2_root)}')

In [ ]:
x_r = np.linspace(-1, 1, 100)
y_n = n0 - (n2*r**2)/2
fig, ax = plt.subplots(figsize=(10,6))
ax.set_xlabel('Radial distance, m')
ax.set_ylabel('IOR')
ax.grid(True, linestyle=':')
ax.plot(x_r, y_n)

In [ ]:
print(f'L={}')
L_rod = L
n2_this = n2
x_n0 = np.linspace(1, 2.5, 100)
y_F = 1/x_n0*np.sqrt(x_n0/n2)/np.tan(np.sqrt(n2/x_n0)*L)
fig, ax = plt.subplots(figsize=(10,6))
ax.set_xlabel('$n_0$')
ax.set_ylabel('Focal range, m')
ax.grid(True, linestyle=':')
ax.plot(x_n0, y_F)

In [ ]:
print(f'n0={n0}, n2={n2}')
x_L = np.linspace(0.01, 1, 100)
y_F = 1/x_n0*np.sqrt(x_n0/n2)/np.tan(np.sqrt(n2/x_n0)*L)
fig, ax = plt.subplots(figsize=(10,6))
ax.set_xlabel('Rod length, m')
ax.set_ylabel('Focal range, m')
ax.grid(True, linestyle=':')
ax.plot(x_L, y_F)